# Birthday paradox

https://en.wikipedia.org/wiki/Birthday_problem

In [41]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
from IPython.display import display

%matplotlib inline

def birthday_paradox(people, trials):
    buffer = np.zeros(people)
    for t in range(trials):
        for p in range(people):
            sample = np.random.choice(np.arange(365), size=p)
            if len(sample) != len(np.unique(sample)):
                buffer[p] += 1.0
    result = buffer / trials
    
    plt.figure(figsize=(8, 5))
    plt.plot(np.arange(people), result, c='k', lw=2)
    plt.title('Probability of at least two people sharing a birthday versus the number of people')
    plt.xlabel('Number of people')
    plt.ylabel('Probability of a pair')
    plt.grid(True)
    plt.show()
    
    return (np.arange(people), result)

In [42]:
p = interactive(birthday_paradox, people=(1,100), trials=(1,1000))
display(p)